<a href="https://colab.research.google.com/github/fuaadrahman/Fruit-Recogniser/blob/main/Fruit_App.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
import tensorflow as tf
import numpy as np
from PIL import Image
import io
import os
from IPython.display import display, HTML
from google.colab import output
import base64

# Define the class names in the correct order.
# This must match the order in which the classes were loaded during training.
CLASS_NAMES = [
    'Apple Braeburn', 'Apple Granny Smith', 'Apricot', 'Avocado', 'Banana',
    'Blueberry', 'Cactus fruit', 'Cantaloupe', 'Cherry', 'Clementine', 'Corn',
    'Cucumber Ripe', 'Grape Blue', 'Kiwi', 'Lemon', 'Limes', 'Mango',
    'Onion White', 'Orange', 'Papaya', 'Passion Fruit', 'Peach', 'Pear',
    'Pepper Green', 'Pepper Red', 'Pineapple', 'Plum', 'Pomegranate',
    'Potato Red', 'Raspberry', 'Strawberry', 'Tomato', 'Watermelon'
]

# --- 1. Load the Model ---
# Make sure your trained model file 'best_model.h5' is accessible in your Colab environment.
model = None
MODEL_PATH = 'best_model.h5'

if not os.path.exists(MODEL_PATH) or os.path.getsize(MODEL_PATH) == 0:
    print(f"Error: Model file '{MODEL_PATH}' not found or is empty.")
    print("Please ensure you have run the model training and saved the file in this Colab session.")
else:
    try:
        model = tf.keras.models.load_model(MODEL_PATH)
        print("Model loaded successfully!")
    except Exception as e:
        print(f"Error loading model: {e}")

# --- 2. Preprocessing and Prediction Functions ---
def preprocess_image(image_bytes):
    """Resizes and normalizes an image for model input."""
    image = Image.open(io.BytesIO(image_bytes)).resize((224, 224))
    image_array = np.array(image, dtype=np.float32)
    image_array = np.expand_dims(image_array, axis=0)
    return image_array / 255.0

def predict_image(image_bytes_string):
    """Predicts the class of an image and returns the result."""
    if model is None:
        return "Error: Model not loaded. Please check the console."

    try:
        # Decode the base64 string back into a bytes object
        image_bytes = base64.b64decode(image_bytes_string)

        processed_image = preprocess_image(image_bytes)
        predictions = model.predict(processed_image, verbose=0)
        predicted_class_index = np.argmax(predictions, axis=1)[0]
        confidence = np.max(predictions)

        predicted_name = CLASS_NAMES[predicted_class_index]

        # Return a formatted string with the prediction and confidence
        return f"Prediction: <b>{predicted_name}</b><br>Confidence: <b>{confidence:.2f}</b>"
    except Exception as e:
        return f"Prediction Error: {e}"

# --- 3. Create the Interactive UI ---
def create_ui():
    """Generates the HTML and JavaScript for the file upload UI."""
    # Register the Python function to be called from JavaScript
    output.register_callback('run_prediction_python', predict_image)

    html_code = """
    <style>
        .container {
            font-family: Arial, sans-serif;
            padding: 20px;
            border-radius: 10px;
            background-color: #f0f0f0;
            text-align: center;
        }
        h2 { color: #333; }
        #image-preview {
            max-width: 100%;
            height: auto;
            border: 2px solid #ccc;
            border-radius: 8px;
            margin-top: 15px;
            display: none;
        }
        .file-upload {
            padding: 10px 20px;
            background-color: #4285F4;
            color: white;
            border: none;
            border-radius: 5px;
            cursor: pointer;
            font-size: 16px;
        }
        .file-upload:hover {
            background-color: #357ae8;
        }
        #result-box {
            margin-top: 20px;
            font-size: 18px;
            background-color: white;
            border: 1px solid #ddd;
            padding: 15px;
            border-radius: 8px;
            min-height: 50px;
        }
    </style>
    <div class="container">
        <h2>🍎 Fruit Recognition App</h2>
        <p>Click the button below to upload a fruit image.</p>
        <button class="file-upload" onclick="upload_and_predict()">
            Upload Image
        </button>
        <img id="image-preview" src="#" alt="Uploaded Image">
        <div id="result-box"></div>
    </div>
    <script>
        // JavaScript to handle the prediction
        async function upload_and_predict() {
            const resultBox = document.getElementById('result-box');
            resultBox.innerHTML = "<i>Processing...</i>";

            const file_input = document.createElement('input');
            file_input.setAttribute('type', 'file');
            file_input.setAttribute('accept', 'image/*');
            file_input.click();

            file_input.onchange = async () => {
                const file = file_input.files[0];
                if (!file) {
                    resultBox.innerHTML = "<i>Upload canceled.</i>";
                    return;
                }

                const reader = new FileReader();
                reader.onload = async (e) => {
                    const imageData = e.target.result;
                    const imagePreview = document.getElementById('image-preview');
                    imagePreview.src = imageData;
                    imagePreview.style.display = 'block';

                    const base64_image = imageData.split(',')[1];

                    try {
                        const response = await google.colab.kernel.invokeFunction('run_prediction_python', [base64_image], {});
                        resultBox.innerHTML = response.data["text/plain"];
                    } catch (error) {
                        resultBox.innerHTML = `An unexpected error occurred: ${error}`;
                        console.error("Colab kernel invocation failed:", error);
                    }
                };
                reader.readAsDataURL(file);
            };
        }
    </script>
    """

    display(HTML(html_code))

# --- Main Execution ---
if __name__ == "__main__":
    create_ui()


Model loaded successfully!
